In [ ]:
import Pkg; Pkg.activate(".")
using FRESCO
import FUSE, QED, IMAS, VacuumFields
using Plots, ProgressMeter

In [ ]:
recycle = false
if recycle
    dd = IMAS.json2imas("./dd_iter.json")
else
    dd, ini, act = FUSE.init(:ITER, init_from=:ods);
    FUSE.ActorEquilibrium(dd, act; ip_from=:pulse_schedule);
    IMAS.imas2json(dd, "./dd_iter.json");
end;

In [ ]:
canvas0 = FRESCO.Canvas(dd, 65)
profile0 = FRESCO.PprimeFFprime(dd);

In [ ]:
# Solve equilibrium with shape control to get self-consistent coil currents
canvas, profile = deepcopy(canvas0), deepcopy(profile0)
control = :shape
# Keep currents fixed in ITER OH coils (1:6), VS coils(13:14), and vacuum vessel(15:162)
fixed_coils = vcat(1:6, 13:14, 15:162) 
@time FRESCO.solve!(canvas, profile, 100, 3; control, relax=0.5, tolerance=1e-12, debug=true, initialize_current=true, fixed_coils);
plot(canvas, size=(500,300))

In [ ]:
# Perturb the VS3U coil current by 1 kA (starts at 0)
canvas1, profile1 = deepcopy(canvas), deepcopy(profile)
dI_13 = 1e3
VacuumFields.set_current!(canvas1.coils[13], dI_13)
@time FRESCO.solve!(canvas1, profile1, 100, 3; relax=0.5, tolerance=1e-6, debug=true, control=:eddy, initialize_current=false, initialize_mutuals=true);

In [ ]:
println("Shift = $(round(canvas1.Zaxis - canvas.Zaxis, digits=5)) m")
p1 = plot(canvas, xrange=(1,13), yrange=(-8.5,8.5), title="Initial", clims=(-160,160))
p2 = plot(canvas1, xrange=(1,13), yrange=(-8.5,8.5), title="Shifted with Eddys", clims=(-160,160), ylabel="")
plot(p1,p2, size=(800,450),left_margin=4Plots.mm)

In [ ]:
# This creates a QED Build struct based on a FRESCO canvas.coils
# The voltage waveforms are set to be constant values that match V=IR
function Build(canvas)
    coils = canvas.coils
    Mcc = [VacuumFields.mutual(c1, c2) for c1 in coils, c2 in coils]
    Ic = [VacuumFields.current(c) / VacuumFields.turns(c) for c in coils] #current per turn
    Rc = [VacuumFields.resistance(c) for c in coils];
    Vc = Ic .* Rc
    V_waveforms = [QED.Waveform{Float64}(t -> Vc[k]) for k in eachindex(Vc)]
    return QED.QED_build(Ic, Vc, Rc, Mcc, V_waveforms)
end

In [ ]:
canvas_save = deepcopy(canvas);

In [ ]:
Δt = 1e-3
Nt = 800
canvas = deepcopy(canvas_save)
VacuumFields.set_current!(canvas.coils[13], dI_13)
build = Build(canvas);
#canvas, profile, build = deepcopy(canvas2), deepcopy(profile2), deepcopy(build2)
#FRESCO.solve!(canvas, profile, 100, 3; relax=0.5, tolerance=1e-6, debug=true, control=:shape);

ps = Plots.Plot{Plots.GRBackend}[]
Zs = zeros(Nt+1)
Zs[1] = canvas.Zaxis
Is = zeros(length(canvas.coils), Nt*1000 + 1)
@showprogress for k in 1:Nt
    #print("$k ")
    for (k, coil) in enumerate(canvas.coils)
        VacuumFields.set_current!(coil, build.Ic[k] * VacuumFields.turns(coil))
    end
    k>1 && FRESCO.set_flux_at_coils!(canvas)
    try
        FRESCO.solve!(canvas, profile, 100, 3; relax=0.5, tolerance=1e-4, debug=false, control=:eddy, initialize_current=false, initialize_mutuals=(k==1));
    catch e
        if typeof(e) <: MethodError
            # The plasma no longer exists
            Zs[k+1:end] .= canvas.Zaxis
            Is[:, ((k-1)* 1000):end] .= Is[:, ((k-1)* 1000) - 1]
            break
        else
            rethrow(e)
        end
    end
    
    for (k, coil) in enumerate(canvas.coils)
        build.Ic[k] = VacuumFields.current(coil) / VacuumFields.turns(coil)
    end

    #R0, Z0, Ip, Vz, Ic = compute_inputs()
    #build.Vs = Voltages_from_Luke(R0, Z0, Ip, Vz, Ic)
    Zs[k+1] = canvas.Zaxis
    
    # UPDATE TO EVOLVE COUPLED SYSTEM
    Is[:, (k-1)* 1000 .+ (1:1001)] .= QED.evolve!(build, Δt, 1000)
    #@show build.Ic
    
    push!(ps, plot(canvas, xrange=(0.75,2.75), yrange=(-2,2), colorbar=false))
    #mod(k, 10) == 0 && 
    #display(plot(canvas, title = "t = $(k * Δt) s"))
end
println()


In [ ]:
for (k,p) in enumerate(ps)
    plot!(p, title="t=$(round(1e3k*Δt)) ms", xlabel="", ylabel="", link=:both, xrange=(1,13),yrange=(-8,8))
end
plot(ps[vcat(10:100:end)]..., layout=(2,4), size=(1200,800))
#plot(ps[1:11]..., size=(1200,800))

In [ ]:
pz = plot(range(0.0, Nt * Δt * 1000., Nt+1), Zs, xlabel="t (ms)", ylabel="ΔZ (m)", lw=2, legend=nothing)